In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
import os
from torch.utils.data import Dataset, DataLoader, random_split
import torch
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets

In [4]:
# Check if GPU is available and set device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the transforms
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [5]:
# Create the dataset
trainset = datasets.ImageFolder("./PokemonData", transform=transform)
dataset_length = len(trainset)

# Split the dataset into training and validation sets
training_data, validation_data = random_split(trainset, [int(0.8 * dataset_length), dataset_length - int(0.8 * dataset_length)])

print(len(trainset.classes))


150


In [6]:
# Create data loaders
trainloader = DataLoader(training_data, batch_size=64, shuffle=True, num_workers=4)
testloader = DataLoader(validation_data, batch_size=64, shuffle=True, num_workers=4)

In [7]:

# Train the model
num_epochs = 50

best_acc = 0.0  # Variable to store the best accuracy on the validation set


In [8]:
# Load the pretrained AlexNet model
model = models.alexnet(pretrained=True)

# Modify the classifier to match the number of classes
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(trainset.classes))

# Move the model to GPU
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # Move inputs and labels to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    # Training accuracy for the epoch
    train_accuracy = correct_train / total_train

    # Evaluate the model on the validation set
    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted_val = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()

    # Validation accuracy for the epoch
    val_accuracy = correct_val / total_val

    print(f'Epoch {epoch + 1}, Training Accuracy: {100 * train_accuracy:.8f}% ({correct_train}/{total_train}), Validation Accuracy: {100 * val_accuracy:.8f}% ({correct_val}/{total_val})')

    # Save the model if it has the best accuracy so far
    if val_accuracy > best_acc:
        best_acc = val_accuracy
        torch.save(model.state_dict(), 'best_model_alexnet.pth')

print('Finished Training')

Epoch 1, Training Accuracy: 23.53372434% (1284/5456), Validation Accuracy: 62.31671554% (850/1364)
Epoch 2, Training Accuracy: 72.67228739% (3965/5456), Validation Accuracy: 74.12023460% (1011/1364)
Epoch 3, Training Accuracy: 85.75879765% (4679/5456), Validation Accuracy: 79.76539589% (1088/1364)
Epoch 4, Training Accuracy: 91.64222874% (5000/5456), Validation Accuracy: 79.76539589% (1088/1364)
Epoch 5, Training Accuracy: 95.36290323% (5203/5456), Validation Accuracy: 79.32551320% (1082/1364)
Epoch 6, Training Accuracy: 95.98607038% (5237/5456), Validation Accuracy: 81.15835777% (1107/1364)
Epoch 7, Training Accuracy: 97.04912023% (5295/5456), Validation Accuracy: 81.37829912% (1110/1364)
Epoch 8, Training Accuracy: 97.96554252% (5345/5456), Validation Accuracy: 82.25806452% (1122/1364)
Epoch 9, Training Accuracy: 98.55205279% (5377/5456), Validation Accuracy: 82.55131965% (1126/1364)
Epoch 10, Training Accuracy: 98.47873900% (5373/5456), Validation Accuracy: 82.18475073% (1121/1364)


In [9]:
best_acc = 0.0  # Variable to store the best accuracy on the validation set
# Load the pretrained AlexNet model
model = models.vgg16(pretrained=True)

# Modify the classifier to match the number of classes
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(trainset.classes))

# Move the model to GPU
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # Move inputs and labels to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    # Training accuracy for the epoch
    train_accuracy = correct_train / total_train

    # Evaluate the model on the validation set
    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted_val = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()

    # Validation accuracy for the epoch
    val_accuracy = correct_val / total_val

    print(f'Epoch {epoch + 1}, Training Accuracy: {100 * train_accuracy:.8f}% ({correct_train}/{total_train}), Validation Accuracy: {100 * val_accuracy:.8f}% ({correct_val}/{total_val})')


    # Save the model if it has the best accuracy so far
    if val_accuracy > best_acc:
        best_acc = val_accuracy
        torch.save(model.state_dict(), 'best_model_vgg16.pth')

print('Finished Training')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\user/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Epoch 1, Training Accuracy: 3.07917889% (168/5456), Validation Accuracy: 19.42815249% (265/1364)
Epoch 2, Training Accuracy: 41.47727273% (2263/5456), Validation Accuracy: 75.07331378% (1024/1364)
Epoch 3, Training Accuracy: 77.69428152% (4239/5456), Validation Accuracy: 82.62463343% (1127/1364)
Epoch 4, Training Accuracy: 87.40835777% (4769/5456), Validation Accuracy: 84.01759531% (1146/1364)
Epoch 5, Training Accuracy: 92.02712610% (5021/5456), Validation Accuracy: 85.26392962% (1163/1364)
Epoch 6, Training Accuracy: 95.12463343% (5190/5456), Validation Accuracy: 87.46334311% (1193/1364)
Epoch 7, Training Accuracy: 96.20601173% (5249/5456), Validation Accuracy: 87.09677419% (1188/1364)
Epoch 8, Training Accuracy: 96.97580645% (5291/5456), Validation Accuracy: 86.43695015% (1179/1364)
Epoch 9, Training Accuracy: 97.58064516% (5324/5456), Validation Accuracy: 87.53665689% (1194/1364)
Epoch 10, Training Accuracy: 98.29545455% (5363/5456), Validation Accuracy: 87.75659824% (1197/1364)
Ep